<h1><strong>Logistic Regression </strong></h1>
<h3> La regresión logística es un modelo linear que funciona calculando, mediante una funcion logistica, las probabilidades de que un evento binario ocurra o no. </h3>
<br> Importo los modulos necesarios

In [5]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.naive_bayes  import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import f1_score

from feature_builder import process_dataset, add_text_embeddings, calculate_keyword_encoding
from hyperparameter_tuning import random_search

<h3>Cargo los datos necesarios

In [6]:
train_dataset = pd.read_csv('train.csv')

In [7]:
test_dataset = pd.read_csv('test.csv')

In [8]:
y = train_dataset.loc[:,'target']

<h4 style="color:red">falta ponerle los hiperparametros

In [9]:
logisticRegr = LogisticRegression()

<h2><strong>Declaro las distintas combinaciones de features

<h3>Tf-idf

In [10]:
x_tfidf = process_dataset(train_dataset, text_type='tfidf', use_spacy=True)

In [11]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x_tfidf, y, test_size = .30, random_state = 17)

<h3> Procesadas con spacy y embeddings y features manuales

In [ ]:
x_processed = process_dataset(train_dataset, use_spacy=True)

In [ ]:
x_train_processed, x_test_processed, y_train_processed, y_test_processed = train_test_split(x_processed, y, test_size = .33, random_state = 17)

<h3>Procesadas solo con embeddings y spacy

In [ ]:
x_embedd = process_dataset(train_dataset, use_manual_features=False, use_spacy=True)

In [ ]:
x_train_embedd, x_test_embedd, y_train_embedd, y_test_embedd = train_test_split(x_embedd, y, test_size = .30, random_state = 17)

<h2><strong>Ahora pruebo los algoritmos

<h3>Primero pruebo con tf-idf

In [ ]:
logisticRegr.fit(x_train_tfidf, y_train_tfidf)

In [ ]:
y_pred_tfidf = logisticRegr.predict(x_test_tfidf)

In [ ]:
f1_score(y_test_tfidf, y_pred_tfidf)

In [ ]:
0.7512588116817723

<h3>Embeddings, spacy y manuales

In [14]:
logisticRegr.fit(x_train_processed, y_train_processed)

/home/andres/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
y_pred_processed = logisticRegr.predict(x_test_processed)

In [16]:
f1_score(y_test_processed, y_pred_processed)

0.7131264916467781

<h3>Embeddings y spacy 

In [28]:
logisticRegr = LogisticRegression(solver='liblinear', penalty='l1', multi_class='auto', max_iter=1000, C=1)

In [29]:
logisticRegr.fit(x_train_embedd, y_train_embedd)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
y_pred_embedd = logisticRegr.predict(x_test_embedd)

In [31]:
f1_score(y_test_embedd, y_pred_embedd)

0.7682539682539683